In [1]:
#import neccassary libraries
import pandas as pd
import numpy as np
from datetime import datetime

import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,TimestampType,VarcharType
from pyspark.sql.functions import *
import requests
from pyspark.sql.functions import regexp_replace, col, concat_ws, lit, substring
import login_key

In [2]:
#Create Spark Session

spark = SparkSession.builder.appName('SrideviCapstone').getOrCreate()

In [3]:
creditcarddf=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user=login_key.username,\
                                     password=login_key.password,\
                                     url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                                     dbtable="creditcard_capstone.cdw_sapp_credit_card").load()

In [4]:
branchdf=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user=login_key.username,\
                                     password=login_key.password,\
                                     url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                                     dbtable="creditcard_capstone.cdw_sapp_branch").load()

In [5]:
custinfodf=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user=login_key.username,\
                                     password=login_key.password,\
                                     url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                                     dbtable="creditcard_capstone.cdw_sapp_customer").load()

In [6]:
custinfodf.show(5)

+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no| FULL_STREET_ADDRESS|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|   CUST_PHONE|         CUST_EMAIL|       LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|Main Street North...|     Natchez|        MS|United States|   39120|(123)123-7818|AHooper@example.com|2018-04-21 11:49:02|
|123453023|      Etta|    brendan|   Holman|4210653310102868|  Redwood Drive, 829|Wethersfield|        CT|United States|    6109|(123)123-8933|EHolman@example.com|2018-04-21 11:49:02|
|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|12th Street East,..

In [ ]:
branchdf.show()

In [ ]:
branchdf.columns

In [ ]:
state_code = input("Please input 2 letters STATE code: ").upper()
output213_df =  creditcarddf.join(branchdf, creditcarddf.BRANCH_CODE == branchdf.BRANCH_CODE)\
                        .filter(col("BRANCH_STATE") == state_code)\
                        .groupby("BRANCH_STATE")\
                        .agg(
                            count(creditcarddf.BRANCH_CODE).alias("Number of Branches"),
                            count(creditcarddf.TRANSACTION_VALUE).alias("Number_of_Transaction"),
                            round(sum(creditcarddf.TRANSACTION_VALUE),2).alias("Total_Value_of_Transaction")
                            )

In [ ]:
output213_df.show(5)

In [ ]:
creditcarddf.show(5)


In [ ]:
creditcarddf.columns


In [ ]:
zipcode = input("Please enter zip code: ")
year_month = input("Enter Year: ") + input("Enter Month: ") + "%"
print("The transactions made by the customers living in the zip code: ", zipcode)

output211_df= creditcarddf.join(custinfodf, creditcarddf.CUST_CC_NO == custinfodf.Credit_card_no)\
                        .select("TRANSACTION_ID", "TRANSACTION_VALUE","TRANSACTION_TYPE", col("TIMEID").alias("Transaction Dates"))\
                        .filter((col("CUST_ZIP") == zipcode ) & (creditcarddf.TIMEID.like(year_month)))\
                        .dropDuplicates()\
                        .sort(col("TIMEID").desc())
output211_df.show()

In [ ]:
ssn_user = "%"+ input("Please enter the last 4 digits of your SSN: ")
name_user = input("Enter your Last Name: ") 
card_user = "%" + input("Enter the last 4 digits of your Credit card Number: ")

output221_df= custinfodf.filter((col("LAST_NAME") == name_user) & custinfodf.SSN.like(ssn_user) & custinfodf.Credit_card_no.like(card_user)).dropDuplicates()                         

output221_df.show()

In [ ]:
ssn_user = input("Please provide your SSN: ")

print("I am happy to help you here, What do you want to update?\n")

options_to_update = input("Please select from the options: \n 1. Address update, 2.Phone Update, 3.Email Update")
if(int(options_to_update) == 1):
    print("ADDRESS UPDATE:\n")
    new_full_street_address = input("\nPlease provide your Street address with the apartment number seperated with comma")
    new_cust_city = input("\nPlease provide your City")
    new_cust_state = input("\nPlease provide your State code")
    new_cust_country = input("\nPlease provide your Country")
    new_cust_zip = input("\nPlease provide your zipcode")
elif(int(options_to_update) == 2):
    print("PHONE NUMBER UPDATE")
    new_cust_phone = str(input("\nPlease provide 10 digit phone number"))
elif(int(options_to_update) == 3):
    print("EMAIL UPDATE")
    new_cust_email = input("Please provide a valid email id")

custinfodf = custinfodf.withColumn("FULL_STREET_ADDRESS", when(col("SSN")==ssn_user, new_full_street_address.title()).otherwise(custinfodf.FULL_STREET_ADDRESS))
custinfodf = custinfodf.withColumn("CUST_CITY", when(col("SSN")==ssn_user, new_cust_city.title()).otherwise(custinfodf.CUST_CITY))
custinfodf = custinfodf.withColumn("CUST_STATE", when(col("SSN")==ssn_user, new_cust_state.upper()).otherwise(custinfodf.CUST_STATE))
custinfodf = custinfodf.withColumn("CUST_COUNTRY", when(col("SSN")==ssn_user, new_cust_country.title()).otherwise(custinfodf.CUST_COUNTRY))
custinfodf = custinfodf.withColumn("CUST_ZIP", when(col("SSN")==ssn_user, new_cust_zip).otherwise(custinfodf.CUST_ZIP))


custinfodf = custinfodf.withColumn("CUST_EMAIL", when(col("SSN")==ssn_user, new_cust_email).otherwise(custinfodf.CUST_EMAIL))

custinfodf = custinfodf.withColumn("CUST_PHONE", when(col("SSN")==ssn_user, new_cust_phone).otherwise(custinfodf.CUST_PHONE))
custinfodf = custinfodf.withColumn("CUST_PHONE", when(col("SSN")==ssn_user, concat(lit("("),substring(col("CUST_PHONE"), 1, 3), lit(")"),
                                                              substring(col("CUST_PHONE"), 4, 3), lit("-"),
                                                              substring(col("CUST_PHONE"), 7, 4))).otherwise(custinfodf.CUST_PHONE))

custinfodf = custinfodf.withColumn("LAST_UPDATED", when(col("SSN")== ssn_user , from_unixtime(current_timestamp().cast("long"))).otherwise(custinfodf.LAST_UPDATED))                                                             

custinfodf.show()

In [14]:
ssn_user = input("\nPlease provide your full SSN: ")

print("\nI am happy to help you here, What do you want to update?\n")

options_to_update = input("""\n Please select from the options: 
                          1. Last Name Update, 2. Credit Card Number Update, 3. Address update, 4.Phone Update, 5.Email Update:   """)
print("\n")
if(int(options_to_update) == 1):
    print("LAST NAME UPDATE\n")
    new_last_name = input("\nPlease provide your Last name")
    custinfodf = custinfodf.withColumn("FULL_STREET_ADDRESS", when(col("SSN")==ssn_user, new_full_street_address.title()).otherwise(custinfodf.FULL_STREET_ADDRESS))
    print("ADDRESS UPDATE:\n")
    new_full_street_address = input("\nPlease provide your Street address with the apartment number seperated with comma   ")
    new_cust_city = input("\nPlease provide your City   ")
    new_cust_state = input("\nPlease provide your State code   ")
    new_cust_country = input("\nPlease provide your Country    ")
    new_cust_zip = input("\nPlease provide your zipcode    ")

    custinfodf = custinfodf.withColumn("FULL_STREET_ADDRESS", when(col("SSN")==ssn_user, new_full_street_address.title()).otherwise(custinfodf.FULL_STREET_ADDRESS))
    custinfodf = custinfodf.withColumn("CUST_CITY", when(col("SSN")==ssn_user, new_cust_city.title()).otherwise(custinfodf.CUST_CITY))
    custinfodf = custinfodf.withColumn("CUST_STATE", when(col("SSN")==ssn_user, new_cust_state.upper()).otherwise(custinfodf.CUST_STATE))
    custinfodf = custinfodf.withColumn("CUST_COUNTRY", when(col("SSN")==ssn_user, new_cust_country.title()).otherwise(custinfodf.CUST_COUNTRY))
    custinfodf = custinfodf.withColumn("CUST_ZIP", when(col("SSN")==ssn_user, new_cust_zip).otherwise(custinfodf.CUST_ZIP))
    
elif(int(options_to_update) == 2):
    print("PHONE NUMBER UPDATE")
    new_cust_phone = str(input("\nPlease provide 10 digit phone number   "))
    custinfodf = custinfodf.withColumn("CUST_PHONE", when(col("SSN")==ssn_user, new_cust_phone).otherwise(custinfodf.CUST_PHONE))
    custinfodf = custinfodf.withColumn("CUST_PHONE", when(col("SSN")==ssn_user, concat(lit("("),substring(col("CUST_PHONE"), 1, 3), lit(")"),
                                                            substring(col("CUST_PHONE"), 4, 3), lit("-"),
                                                            substring(col("CUST_PHONE"), 7, 4))).otherwise(custinfodf.CUST_PHONE))

elif(int(options_to_update) == 3):
    print("EMAIL UPDATE")
    new_cust_email = input("Please provide a valid email id    ")
    custinfodf = custinfodf.withColumn("CUST_EMAIL", when(col("SSN")==ssn_user, new_cust_email).otherwise(custinfodf.CUST_EMAIL))
    

custinfodf = custinfodf.withColumn("LAST_UPDATED", when(col("SSN")== ssn_user , from_unixtime(current_timestamp().cast("long"))).otherwise(custinfodf.LAST_UPDATED))                                                             

display(custinfodf.sort(desc("LAST_UPDATED")).show(1))



I am happy to help you here, What do you want to update?



EMAIL UPDATE
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------+-------------+-------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no| FULL_STREET_ADDRESS|CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|   CUST_PHONE|   CUST_EMAIL|       LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------+-------------+-------------------+
|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|12th Street East,...|  Huntley|        IL|United States|   60142|(123)124-3018|123@gmail.com|2023-10-12 17:08:38|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------+-------------+-------------------+
only showing top 1 row



None

In [10]:
ssn_user = "%"+ input("Please enter the last 4 digits of your SSN: ")
name_user = input("Enter your Last Name: ") 
last_4_digits = input("Enter the last 4 digits of your Credit card Number: ")
card_user = "%" + last_4_digits

billyear = input("Enter the Year : ")
billmonth = input("Enter the Month: ")
#print(billmonth)
if (int(billmonth)>=10) or (len(billmonth)==2):
        curr_month = billyear + billmonth + "%"
        prev_month = billyear + "0"+str(int(billmonth)-1) + "%"
        monthlist = billmonth
else:
        prev_month = billyear + "0" +str(int(billmonth)-1) + "%"
        curr_month = billyear + "0"+ billmonth + "%"
        month_list = "0"+billmonth

months_list = {"01":"January","02":"Feburary", 
            "03":"March", "04":"April", 
            "05":"May", "06":"June", 
            "07":"July", "08":"August",
            "09":"September", "10":"October",
            "11":"November", "12":"December"}

firstname = custinfodf.filter((col("LAST_NAME") == name_user.title()) & custinfodf.SSN.like(ssn_user) & custinfodf.Credit_card_no.like(card_user)).head()[1]
lastname = custinfodf.filter((col("LAST_NAME") == name_user.title()) & custinfodf.SSN.like(ssn_user) & custinfodf.Credit_card_no.like(card_user)).head()[3]
credit_card_number = custinfodf.filter((col("LAST_NAME") == name_user.title()) & custinfodf.SSN.like(ssn_user) & custinfodf.Credit_card_no.like(card_user)).head()[4]
                                                                          
print("\n")
print("                           ACCOUNT SUMMARY                                                  ")
print(f"Monthly Bill for the Credit Card Holder *******{last_4_digits} ---- {firstname} {lastname} ")

print(f"Billing Period: {months_list[billmonth]} {billyear}")
print("Previous Balance:                            ")
output221_df_prev = creditcarddf.filter((col("CUST_CC_NO") == credit_card_number) & (creditcarddf.TIMEID.like(prev_month)))\
        .withColumn("Date", to_date(col("TIMEID"), "yyyyMMdd"))\
        .select("Date", "TRANSACTION_TYPE", "TRANSACTION_VALUE", "TRANSACTION_ID")
prev_month_balance = output221_df_prev.agg(round(sum(creditcarddf.TRANSACTION_VALUE),2).alias("PREVIOUS BALANCE"))
prev_month_balance.show()
output221_df_curr = creditcarddf.filter((col("CUST_CC_NO") == credit_card_number) & (creditcarddf.TIMEID.like(curr_month)))\
        .withColumn("Date", to_date(col("TIMEID"), "yyyyMMdd"))\
        .select("Date", "TRANSACTION_TYPE", "TRANSACTION_VALUE", "TRANSACTION_ID")
print(f"Individual Breakup Summary for the month   {months_list[billmonth]} {billyear}                         \n")
output221_df_curr.show(10)
print("Purchases and Adjustments Balance                            \n")
Purchases_Adjustments = output221_df_curr.agg(round(sum(creditcarddf.TRANSACTION_VALUE),2).alias("PURCHASES AND ADJUSTMENTS"))
Purchases_Adjustments.show()
print("Minimum Payment Due:                            $0\n")
print("Fees Charged:                                   $0\n")
print("Interest Charged:                              $0\n")
print("New Balance:                                       ")
New_Balance = output221_df_curr.agg(round(sum(creditcarddf.TRANSACTION_VALUE),2).alias("NEW ACCOUNT BALANCE"))
New_Balance.show()





                           ACCOUNT SUMMARY                                                  
Monthly Bill for the Credit Card Holder *******2868 ---- Etta Holman 
Billing Period: April 2018
Previous Balance:                            
+----------------+
|PREVIOUS BALANCE|
+----------------+
|           19.33|
+----------------+

Individual Breakup Summary for the month   April 2018                         

+----------+----------------+-----------------+--------------+
|      Date|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------+----------------+-----------------+--------------+
|2018-04-16|       Education|            10.73|         36784|
|2018-04-11|           Bills|            26.35|         36787|
|2018-04-18|      Healthcare|            87.23|         36793|
+----------+----------------+-----------------+--------------+

Purchases and Adjustments Balance                            

+-------------------------+
|PURCHASES AND ADJUSTMENTS|
+------------------------

In [ ]:
New_Balance = output221_df_curr.agg(round(sum(creditcarddf.TRANSACTION_VALUE),2).alias("NEW ACCOUNT BALANCE"))
print(New_Balance)

In [ ]:
ssn_user = "%"+ input("Please enter the last 4 digits of your SSN: ")
last_4_digits = input("Enter the last 4 digits of your Credit card Number: ")
card_user = "%" + last_4_digits

start_date = (input("Enter Starting date YYYYMMDD format: "))
end_date = (input("Enter Ending date YYYYMMDD format: "))
output_224df = creditcarddf.filter((col("TIMEID") > start_date ) & (col("TIMEID") < end_date ))\
                        .filter(creditcarddf.CUST_SSN.like(ssn_user) & creditcarddf.CUST_CC_NO.like(card_user))\
                        .withColumn("Date", to_date(col("TIMEID"), "yyyyMMdd"))\
                        .sort(col("TIMEID").desc())\
                        .select( "Date", "BRANCH_CODE", "TRANSACTION_TYPE", "TRANSACTION_VALUE","TRANSACTION_ID")
                      
print(f"The Transactions happened between {end_date} (YYYYMMDD) and {start_date} (YYYYMMDD) for the customer having the credit card ending with last 4 digits {last_4_digits}:")
print("\n")
output_224df.show()

In [22]:
ssn_user = "%"+ input("Please enter the last 4 digits of your SSN: ")
name_user = input("Enter your Last Name: ") 
#card_user = "%" + input("Enter the last 4 digits of your Credit card Number: ")

#output221_df= custinfodf.filter((col("LAST_NAME") == name_user) & custinfodf.SSN.like(ssn_user) & custinfodf.Credit_card_no.like(card_user)).dropDuplicates()
output221_df= custinfodf.filter((col("LAST_NAME") == name_user) & custinfodf.SSN.like(ssn_user)).dropDuplicates()

output221_df.show()


+---------+----------+-----------+---------+----------------+-------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no|FULL_STREET_ADDRESS|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|   CUST_PHONE|         CUST_EMAIL|       LAST_UPDATED|
+---------+----------+-----------+---------+----------------+-------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|123453023|      Etta|    brendan|   Holman|4210653310102868| Redwood Drive, 829|Wethersfield|        CT|United States|    6109|(123)123-8933|EHolman@example.com|2018-04-21 11:49:02|
+---------+----------+-----------+---------+----------------+-------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+

